In [11]:
import os
import cv2
import numpy as np
from PIL import Image
import sys
from tools import extract_frames, get_cache_video, uniform_sample, get_video_total_frames, keyframes_sampler
import subprocess
import tempfile


def Un_sampler(file_idx, video_path, args=None):
    # 用于均匀采样13帧
    if args:
        process_dataset = args.process_dataset
    else:
        process_dataset = 'hd3m'
    
    image_name_list = []
    image_dict_list = []
    
    if process_dataset == 'webvid':
        video_prefix = "s3://vision-language-data/video-data/webvid10m/process_videos/"
    elif process_dataset == 'hd3m':
        video_prefix = "s3://vision-language-data/video-data/hd130m/process_videos/"
    video_path = os.path.join(video_prefix, video_path)
    cache_path = get_cache_video(video_path)
    images = extract_frames(cache_path)
    os.remove(cache_path)

    for index, img in enumerate(images):
        image_name_list.append(f"{file_idx:09d}-{index}")
        image_dict_list.append(dict(
            __key__=f"{file_idx:09d}-{index}",
            jpg=img,
        ))
    
    return image_name_list, image_dict_list

def KF_sampler(file_idx, video_path, args=None):
    #  Only supported for InternVid dataset, whose video is local
    I_images,I_indices,I_total_frames = keyframes_sampler(video_path, 'I', max_samples = args.Iframes)
    len_PFrames = args.max_frames - len(I_images)
    P_images,P_indices,P_total_frames = keyframes_sampler(video_path, 'P', max_samples = len_PFrames)
    imgages = combineKeyFrames(I_images, P_images, I_indices, P_indices)
    
    image_name_list = []
    image_dict_list = []
    
    for index, img in enumerate(images):
        image_name_list.append(f"{file_idx:09d}-{index}")
        image_dict_list.append(dict(
            __key__=f"{file_idx:09d}-{index}",
            jpg=img,
        ))
    
    return image_name_list, image_dict_list



class args:
    Iframes = 8
    max_frames = 24

file_idx = 0
video_path = "/data/webvid/test.mp4"
I_images, I_indices, P_images, P_indices = KF_sampler(file_idx, video_path, args=args)
new_images, new_indices = interleave_images(I_images, I_indices, P_images, P_indices)